In [1]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import gradio as gr
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import openai
import requests
import json
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [5]:
class FinancialAnalyzer:
    def __init__(self, OPENAI_API_KEY: str):
         self.openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [6]:
def fetch_stock_data(self, symbol: str, period: str = "1y") -> pd.DataFrame:
        """Fetch stock data using yfinance"""
        try:
            ticker = yf.Ticker(symbol.upper())
            data = ticker.history(period=period)
            return data
        except Exception as e:
            raise Exception(f"Error fetching stock data: {str(e)}")

In [7]:
def fetch_crypto_data(self, symbol: str, days: int = 365) -> pd.DataFrame:
        """Fetch cryptocurrency data using CoinGecko API"""
        try:
            url = f"https://api.coingecko.com/api/v3/coins/{symbol.lower()}/market_chart"
            params = {
                'vs_currency': 'usd',
                'days': days,
                'interval': 'daily'
            }
            response = requests.get(url, params=params)
            data = response.json()
            
            # Convert to DataFrame
            prices = data['prices']
            df = pd.DataFrame(prices, columns=['timestamp', 'price'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            df.rename(columns={'price': 'Close'}, inplace=True)
            
            return df
        except Exception as e:
            raise Exception(f"Error fetching crypto data: {str(e)}")

In [8]:
 def create_price_chart(self, data: pd.DataFrame, symbol: str, asset_type: str) -> plt.Figure:
        """Create a price trend chart using Matplotlib"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        # Price chart
        ax1.plot(data.index, data['Close'], linewidth=2, color='#2E86C1')
        ax1.set_title(f'{symbol.upper()} - {asset_type} Price Trend', fontsize=16, fontweight='bold')
        ax1.set_ylabel('Price (USD)', fontsize=12)
        ax1.grid(True, alpha=0.3)
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
        
        # Moving averages if stock data
        if asset_type == "Stock" and len(data) > 50:
            data['MA20'] = data['Close'].rolling(window=20).mean()
            data['MA50'] = data['Close'].rolling(window=50).mean()
            ax1.plot(data.index, data['MA20'], label='20-day MA', alpha=0.7, color='orange')
            ax1.plot(data.index, data['MA50'], label='50-day MA', alpha=0.7, color='red')
            ax1.legend()
        
        # Volume chart (if available)
        if 'Volume' in data.columns and not data['Volume'].isna().all():
            ax2.bar(data.index, data['Volume'], alpha=0.6, color='gray')
            ax2.set_title('Trading Volume', fontsize=14)
            ax2.set_ylabel('Volume', fontsize=12)
            ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
        else:
            # Price change visualization
            price_change = data['Close'].pct_change()
            colors = ['green' if x > 0 else 'red' for x in price_change]
            ax2.bar(data.index, price_change * 100, alpha=0.6, color=colors)
            ax2.set_title('Daily Price Change (%)', fontsize=14)
            ax2.set_ylabel('Change (%)', fontsize=12)
            ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        
        plt.tight_layout()
        plt.xticks(rotation=45)
        return fig

In [9]:
def calculate_metrics(self, data: pd.DataFrame) -> Dict:
        """Calculate key financial metrics"""
        current_price = data['Close'].iloc[-1]
        price_change = data['Close'].iloc[-1] - data['Close'].iloc[-2]
        price_change_pct = (price_change / data['Close'].iloc[-2]) * 100
        
        # Historical metrics
        high_52w = data['Close'].max()
        low_52w = data['Close'].min()
        volatility = data['Close'].pct_change().std() * 100
        
        # Recent performance
        returns_7d = ((data['Close'].iloc[-1] / data['Close'].iloc[-7]) - 1) * 100 if len(data) >= 7 else 0
        returns_30d = ((data['Close'].iloc[-1] / data['Close'].iloc[-30]) - 1) * 100 if len(data) >= 30 else 0
        
        return {
            'current_price': current_price,
            'price_change': price_change,
            'price_change_pct': price_change_pct,
            'high_52w': high_52w,
            'low_52w': low_52w,
            'volatility': volatility,
            'returns_7d': returns_7d,
            'returns_30d': returns_30d
        }

In [10]:
def generate_ai_recommendation(self, symbol: str, metrics: Dict, asset_type: str) -> str:
        """Generate AI-powered investment recommendation"""
        try:
            prompt = f"""
            As a financial analyst, provide a comprehensive analysis and recommendation for {symbol.upper()} ({asset_type}).
            
            Current Financial Metrics:
            - Current Price: ${metrics['current_price']:.2f}
            - Daily Change: ${metrics['price_change']:.2f} ({metrics['price_change_pct']:.2f}%)
            - 52-week High: ${metrics['high_52w']:.2f}
            - 52-week Low: ${metrics['low_52w']:.2f}
            - Volatility: {metrics['volatility']:.2f}%
            - 7-day Return: {metrics['returns_7d']:.2f}%
            - 30-day Return: {metrics['returns_30d']:.2f}%
            
            Please provide:
            1. Current market position analysis
            2. Risk assessment
            3. Investment recommendation (Buy/Hold/Sell)
            4. Key factors to watch
            5. Time horizon considerations
            
            Keep the analysis concise but comprehensive, suitable for both novice and experienced investors.
            """
            
            response = self.openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an expert financial analyst providing investment advice based on technical analysis and market data."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=800,
                temperature=0.7
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            return f"AI analysis temporarily unavailable. Error: {str(e)}\n\nBased on the metrics provided, here's a basic analysis:\n- Current price is ${metrics['current_price']:.2f}\n- Daily change: {metrics['price_change_pct']:.2f}%\n- Consider the high volatility of {metrics['volatility']:.2f}% when making investment decisions."


In [11]:
def create_gradio_interface():
    """Create and configure the Gradio interface"""
    

In [12]:
with gr.Blocks(title="Financial Data Extractor & Reporter", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 📈 Financial Data Extractor & Reporter
    
    Analyze stocks and cryptocurrencies with AI-powered insights and beautiful visualizations.
    
    **Features:**
    - Real-time price data fetching
    - Interactive trend visualizations
    - AI-powered investment recommendations
    - Support for stocks and cryptocurrencies
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🔧 Configuration")
            
            openai_key = gr.Textbox(
                label="OpenAI API Key (Optional if using .env)",
                placeholder="Leave empty to use .env file",
                type="password",
                info="Will use .env file if empty"
            )
            
            symbol = gr.Textbox(
                label="Symbol",
                placeholder="AAPL, bitcoin, ethereum",
                value="AAPL",
                info="Stock ticker or crypto name"
            )
            
            asset_type = gr.Radio(
                label="Asset Type",
                choices=["Stock", "Cryptocurrency"],
                value="Stock"
            )
            
            period = gr.Dropdown(
                label="Time Period",
                choices=["1mo", "3mo", "6mo", "1y", "2y"],
                value="1y"
            )
            
            analyze_btn = gr.Button("🔍 Analyze", variant="primary", size="lg")
            
            gr.Markdown("""
            ### 📝 Usage Tips:
            - **Stocks:** Use ticker symbols (AAPL, GOOGL, TSLA)
            - **Crypto:** Use coin names (bitcoin, ethereum, cardano)
            - Your .env file will be automatically loaded
            """)
    
        with gr.Column(scale=2):
            gr.Markdown("### 📊 Analysis Results")
            
            with gr.Tab("📈 Price Chart"):
                price_chart = gr.Plot(label="Price Trend Analysis")
            
            with gr.Tab("📋 Key Metrics"):
                metrics_display = gr.Markdown(label="Financial Metrics")
            
            with gr.Tab("🤖 AI Recommendation"):
                ai_recommendation = gr.Markdown(label="Investment Analysis")


In [13]:
    def analyze_financial_asset(symbol, asset_type, period, openai_key):
        # Use .env file if no key provided via UI
        if not openai_key:
            openai_key = os.getenv('OPENAI_API_KEY')
            if not openai_key:
                return None, "Please provide OpenAI API key via interface or .env file", ""
        
        try:
            analyzer = FinancialAnalyzer(openai_key)
            
            days_map = {"1mo": 30, "3mo": 90, "6mo": 180, "1y": 365, "2y": 730}
            
            # Fetch data
            if asset_type == "Stock":
                data = analyzer.fetch_stock_data(symbol, period)
            else:
                days = days_map.get(period, 365)
                data = analyzer.fetch_crypto_data(symbol, days)
            
            if data.empty:
                return None, f"No data found for {symbol}", ""
            
            # Calculate metrics
            metrics = analyzer.calculate_metrics(data)
            
            # Create plot
            chart = analyzer.create_price_chart(data, symbol, asset_type)
            
            # Generate AI recommendation
            recommendation = analyzer.generate_ai_recommendation(symbol, metrics, asset_type)
            
            # Format metrics
            metrics_text = f"""
            ## 📊 Key Metrics for {symbol.upper()}
            
            **Current Price:** ${metrics['current_price']:.2f}
            **Daily Change:** ${metrics['price_change']:.2f} ({metrics['price_change_pct']:.2f}%)
            **52-Week Range:** ${metrics['low_52w']:.2f} - ${metrics['high_52w']:.2f}
            **Volatility:** {metrics['volatility']:.2f}%
            **7-Day Return:** {metrics['returns_7d']:.2f}%
            **30-Day Return:** {metrics['returns_30d']:.2f}%
            """
            
            return chart, metrics_text, recommendation
            
        except Exception as e:
            return None, f"Error: {str(e)}", ""


In [14]:
import os
from dotenv import load_dotenv
import gradio as gr
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import openai
import requests
import warnings
from typing import Dict

load_dotenv()
warnings.filterwarnings('ignore')


In [15]:
class FinancialAnalyzer:
    def __init__(self, openai_api_key: str):
        self.openai_client = openai.OpenAI(api_key=openai_api_key)

    def fetch_stock_data(self, symbol: str, period: str = "1y") -> pd.DataFrame:
        ticker = yf.Ticker(symbol.upper())
        data = ticker.history(period=period)
        return data

    def fetch_crypto_data(self, symbol: str, days: int = 365) -> pd.DataFrame:
        url = f"https://api.coingecko.com/api/v3/coins/{symbol.lower()}/market_chart"
        params = {'vs_currency': 'usd', 'days': days, 'interval': 'daily'}
        response = requests.get(url, params=params)
        prices = response.json()['prices']
        df = pd.DataFrame(prices, columns=['timestamp', 'price'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df.rename(columns={'price': 'Close'}, inplace=True)
        return df

    def calculate_metrics(self, data: pd.DataFrame) -> Dict:
        # Add your metric calculation logic here as before
        current_price = data['Close'].iloc[-1]
        price_change = data['Close'].iloc[-1] - data['Close'].iloc[-2]
        price_change_pct = (price_change / data['Close'].iloc[-2]) * 100
        high_52w = data['Close'].max()
        low_52w = data['Close'].min()
        volatility = data['Close'].pct_change().std() * 100
        returns_7d = ((data['Close'].iloc[-1] / data['Close'].iloc[-7]) - 1) * 100 if len(data) >= 7 else 0
        returns_30d = ((data['Close'].iloc[-1] / data['Close'].iloc[-30]) - 1) * 100 if len(data) >= 30 else 0
        return {
            'current_price': current_price,
            'price_change': price_change,
            'price_change_pct': price_change_pct,
            'high_52w': high_52w,
            'low_52w': low_52w,
            'volatility': volatility,
            'returns_7d': returns_7d,
            'returns_30d': returns_30d
        }

    def generate_ai_recommendation(self, symbol: str, metrics: Dict, asset_type: str) -> str:
        # Add your AI prompt and response logic as before
        try:
            prompt = f"""As a financial analyst, provide a comprehensive analysis and recommendation for {symbol.upper()} ({asset_type}).\n\nCurrent Price: ${metrics['current_price']:.2f}\nDaily Change: ${metrics['price_change']:.2f} ({metrics['price_change_pct']:.2f}%)\n52-Week Range: ${metrics['low_52w']:.2f} - ${metrics['high_52w']:.2f}\nVolatility: {metrics['volatility']:.2f}%"""
            response = self.openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an expert financial analyst."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=800,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"AI analysis error: {str(e)}"

    def create_price_chart(self, data: pd.DataFrame, symbol: str, asset_type: str) -> plt.Figure:
        # Add your plot logic as before
        fig, ax = plt.subplots(figsize=(12, 5))
        ax.plot(data.index, data['Close'], linewidth=2, color='#2E86C1')
        ax.set_title(f'{symbol.upper()} - {asset_type} Price Trend')
        ax.set_ylabel('Price (USD)')
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        return fig


In [16]:
with gr.Blocks(title="Financial Data Extractor & Reporter", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 📈 Financial Data Extractor & Reporter
    
    Analyze stocks and cryptocurrencies with AI-powered insights and visualizations.
    """)
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🔧 Configuration")
            openai_key = gr.Textbox(
                label="OpenAI API Key (Optional if using .env)",
                placeholder="Leave empty to use .env file",
                type="password",
                info="Will use .env file if empty"
            )
            symbol = gr.Textbox(label="Symbol", placeholder="AAPL, bitcoin, ethereum", value="AAPL")
            asset_type = gr.Radio(label="Asset Type", choices=["Stock", "Cryptocurrency"], value="Stock")
            period = gr.Dropdown(label="Time Period", choices=["1mo", "3mo", "6mo", "1y", "2y"], value="1y")
            analyze_btn = gr.Button("🔍 Analyze", variant="primary", size="lg")
            gr.Markdown("### 📝 Usage Tips:\n- Stocks: AAPL, GOOGL, TSLA\n- Crypto: bitcoin, ethereum, cardano")
        with gr.Column(scale=2):
            gr.Markdown("### 📊 Analysis Results")
            with gr.Tab("📈 Price Chart"):
                price_chart = gr.Plot(label="Price Trend Analysis")
            with gr.Tab("📋 Key Metrics"):
                metrics_display = gr.Markdown(label="Financial Metrics")
            with gr.Tab("🤖 AI Recommendation"):
                ai_recommendation = gr.Markdown(label="Investment Analysis")

    def analyze_financial_asset(symbol, asset_type, period, openai_key):
        if not openai_key:
            openai_key = os.getenv('OPENAI_API_KEY')
            if not openai_key:
                return None, "Please provide OpenAI API key via interface or .env file", ""
        try:
            analyzer = FinancialAnalyzer(openai_key)
            if asset_type == "Stock":
                data = analyzer.fetch_stock_data(symbol, period)
            else:
                days = {"1mo": 30, "3mo": 90, "6mo": 180, "1y": 365, "2y": 730}.get(period, 365)
                data = analyzer.fetch_crypto_data(symbol, days)
            if data.empty:
                return None, f"No data found for {symbol}", ""
            metrics = analyzer.calculate_metrics(data)
            chart = analyzer.create_price_chart(data, symbol, asset_type)
            recommendation = analyzer.generate_ai_recommendation(symbol, metrics, asset_type)
            metrics_text = f"""
            ## 📊 Key Metrics for {symbol.upper()}\n
            **Current Price:** ${metrics['current_price']:.2f}\n
            **Daily Change:** ${metrics['price_change']:.2f} ({metrics['price_change_pct']:.2f}%)\n
            **52-Week Range:** ${metrics['low_52w']:.2f} - ${metrics['high_52w']:.2f}\n
            **Volatility:** {metrics['volatility']:.2f}%\n
            **7-Day Return:** {metrics['returns_7d']:.2f}%\n
            **30-Day Return:** {metrics['returns_30d']:.2f}%\n
            """
            return chart, metrics_text, recommendation
        except Exception as e:
            return None, f"Error: {str(e)}", ""

    analyze_btn.click(
        fn=analyze_financial_asset,
        inputs=[symbol, asset_type, period, openai_key],
        outputs=[price_chart, metrics_display, ai_recommendation]
    )

   


In [ ]:
demo.launch(share=True, debug=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3bc32b3b271e96f759.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Copy the same app.py code from above, but add at the end:

if __name__ == "__main__":
    demo.launch(
        server_name="0.0.0.0",  # Makes it accessible from other devices on your network
        server_port=7860,
        share=False,  # Remove share=True for local hosting
        debug=True
    )
